In [1]:
import firebase_admin
from models import owner
from firebase_admin import firestore, credentials, storage

In [3]:
cred = credentials.Certificate('../creds/firebase_creds.json')

app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [4]:
db.collections('users')

<generator object Client.collections at 0x0000016B7F4C8370>

In [4]:
dotenv.load_dotenv('../ref/.env')

True

In [5]:
league_id = os.environ.get('LEAGUE_ID')

In [63]:
prev_draft_round = {
    '7564' : 1,
    '5846' : 3,
    '3321' : 2,
    '4034' : 1,
    '2133' : 1,
    '2449' : 2,
    '6794' : 1,
    '4046' : 2,
    '6786' : 2,
    '4984' : 4,
    '7591' : 14,
    '8146' : 10,
    '5967' : 8,
    '6797' : 4,
    '6904' : 7,
    '7547' : 5,
    '4881' : 5,
    '4663' : 1,
    '4039' : 1,
    '4866' : 2
}

In [3]:
roster_manager = {
    '1' : 'Mason',
    '2' : 'Conner',
    '3' : 'Catherine',
    '4' : 'Spencer',
    '5' : 'Madelyn',
    '6' : 'Christian',
    '7' : 'Jared',
    '8' : 'Andrew',
    '9' : 'Ryan',
    '10' : 'Uriah'
}

roster_manager_df = pd.DataFrame(roster_manager.items(), columns=['roster_id', 'owner_name'])

In [3]:
def getNflState():
    nfl_url = 'https://api.sleeper.app/v1/state/nfl'
    r = requests.get(nfl_url)
    if r.status_code == 200:
        nfl_state = r.json()
        current_week = nfl_state['week']
        current_season = nfl_state['season']
        week_type = nfl_state['season_type']
        return current_week, current_season, week_type

In [263]:
def getLeagueInfo(league_id: str):
    league_url = f'https://api.sleeper.app/v1/league/{league_id}'
    r = requests.get(league_url)
    if r.status_code == 200:
        league_info = r.json()
        leagues = {}
        leagues[league_info['league_id']] = {
            'total_teams' : league_info['total_rosters'],
            'roster_poositions' : league_info['roster_positions'],
            'draft_id' : league_info['draft_id'],
            'season' : league_info['season'],
            'league_name' : league_info['name'],
            'playoff_start_wk' : league_info['settings']['playoff_week_start'],
            'num_playoff_teams' : league_info['settings']['playoff_teams'],
            'faab_budget' : league_info['settings']['waiver_budget']
            }
        return leagues
    else:
        return None

In [28]:
def getManagers(league_id: str):
    managers_url = f'https://api.sleeper.app/v1/league/{league_id}/users'
    managers_resp = requests.get(managers_url)
    if managers_resp.status_code == 200:
        managers_list = managers_resp.json()
        return pd.DataFrame([(manager['user_id'], manager['display_name']) for manager in managers_list], columns=['manager_id', 'display_name'])
    else:
        return None

In [47]:
managers_df = getManagers(league_id)

In [36]:
def getCurrentRosters(league_id: str):
    rosters_url = f'https://api.sleeper.app/v1/league/{league_id}/rosters'
    rosters_resp = requests.get(rosters_url)
    if rosters_resp.status_code == 200:
        rosters_list = rosters_resp.json()
        return pd.DataFrame([(roster['owner_id'], roster['players']) for roster in rosters_list], columns=['manager_id', 'players']).explode('players')
    else:
        return None


In [48]:
curr_rosters_df = getCurrentRosters(league_id)

In [253]:
def getRosters(league_id: str):
    rosters_url = f'https://api.sleeper.app/v1/league/{league_id}/rosters'
    r = requests.get(rosters_url)
    if r.status_code == 200:
        rosters_resp = r.json()
        rosters = []
        for roster in rosters_resp:
            settings = roster['settings']
            metadata = roster['metadata']
            owner_id = roster['owner_id']
            mngr_req = requests.get(f'https://api.sleeper.app/v1/user/{owner_id}')
            username = None
            if mngr_req.status_code == 200: username = mngr_req.json()['username']
            rosters.append([roster['roster_id'], owner_id, roster_manager[str(roster['roster_id'])], metadata['streak'], metadata['record'], settings['wins'], settings['losses'], settings['ties'], settings['waiver_budget_used']])
        rosters_df = pd.DataFrame(rosters, columns=['roster_id', 'owner_id', 'owner_name', 'current_streak', 'record', 'wins', 'losses', 'ties', 'faab_used'])
        rosters = {}
        for roster in rosters_resp:
            rosters[roster['owner_id']] = {
                'wins' : roster['settings']['wins'],
                'losses' : roster['settings']['losses'],
                'ties' : roster['settings']['ties'],
                'pts_for' : f'{roster['settings']['fpts']}.{roster['settings']['fpts_decimal']}',
                'pts_against' : f'{roster['settings']['fpts_against']}.{roster['settings']['fpts_against_decimal']}',
                'total_moves' : roster['settings']['total_moves'],
                'starters' : roster['starters'],
                'ir' : roster['reserve'],
                'bench' : list(set(roster['players']) - set(roster['starters'] if roster['starters'] is not None else []) - set(roster['reserve'] if roster['reserve'] is not None else []))
                }
        return rosters, rosters_df
    else:
        return None

In [6]:
def getTransactions(league_id: str, week: str):
    transaction_url = f'https://api.sleeper.app/v1/league/{league_id}/transactions/{week}'
    r = requests.get(transaction_url)
    statuses = []
    if r.status_code == 200:
        transactions = r.json()
        succesful_transactions = []
        for transaction in transactions:
            if transaction['status'] == 'complete':
                if transaction['type'] == 'waiver':
                    pass
                elif transaction['type'] == 'free_agent':
                    pass
                elif transaction['type'] == 'trade':
                    pass
                succesful_transactions.append(transaction)
        return succesful_transactions
    else:
        return None

In [59]:
def getMatchups(league_id: str, week: str):
    matchup_url = f'https://api.sleeper.app/v1/league/{league_id}/matchups/{week}'
    r=requests.get(matchup_url)
    matchup_list = []
    players_list = []
    if r.status_code == 200:
        matchups = r.json()
        for team in matchups:
            roster_id = team['roster_id']
            for player in team['players_points']:
                players_list.append([player, roster_id, int(week), True if player in team['starters'] else False, team['players_points'][player]])
            matchup_list.append([team['matchup_id'], int(week), roster_id])
    
    return players_list, matchup_list
        

In [81]:
def getLeagueDraft(league_id: str):
    draft_url = f'https://api.sleeper.app/v1/league/{league_id}/drafts'
    draft_response=requests.get(draft_url)
    if draft_response.status_code == 200:
        draft_dict = draft_response.json()[0]
        draft_id = draft_dict['draft_id']
        draft_picks_url = f'https://api.sleeper.app/v1/draft/{draft_id}/picks'
        draft_picks_resp = requests.get(draft_picks_url)
        if draft_picks_resp.status_code == 200:
            draft_picks = draft_picks_resp.json()
            picks = [(pick['player_id'], pick['round'], pick['picked_by'], pick['is_keeper']) for pick in draft_picks]
            picks_df = pd.DataFrame(picks, columns=['player_id', 'round', 'manager_id', 'is_keeper'])
    return picks_df if not picks_df.empty else None

In [191]:
draft_picks_df = getLeagueDraft(league_id=league_id)

In [ ]:
# TODO: add functionality to pull the highest scoring player available during the draft
def getBestDraftPlayer():
    pass

In [ ]:
# TODO: add playoff chance calculator

In [ ]:
# TODO: add clinching criteria, a la https://theffhub.com/PlayoffMachine?site=sleeper&leagueId=984533715101495296&userId=&s2=&swid=

In [ ]:
# TODO: add schedule comparison

In [228]:
def longest_win_streak(row):
    streaks = row.split('L')
    max_win_streak = max(map(len, streaks))
    return max_win_streak

def longest_loss_streak(row):
    streaks = row.split('W')
    max_loss_streak = max(map(len, streaks))
    return max_loss_streak

In [8]:
def getPlayers(update_players=False):
    if update_players:
        players_url = 'https://api.sleeper.app/v1/players/nfl'
        players_resp = requests.get(players_url)
        if players_resp.status_code == 200:
            players_dict = players_resp.json()
            with open('../ref/players.json') as players_out:
                json.dumps(players_dict, sort_keys=True, indent=4)
    with open('../ref/players.json') as players_in:
        players = json.load(players_in)
    players_list = []
    for player in players:
        position = players[player]['position']
        is_rookie = False
        if position != 'DEF':
            is_rookie = True if players[player]['years_exp'] == 0 else False
        if players[player]['active']: players_list.append([player, players[player]['first_name'], players[player]['last_name'], is_rookie, position])
    return pd.DataFrame(players_list, columns=['player_id', 'first_name', 'last_name', 'is_rookie', 'position'])


In [166]:
players_df = getPlayers()

In [192]:
draft_picks_df['adj_round'] = draft_picks_df['player_id'].apply(lambda x : prev_draft_round.get(x))
draft_picks_df['adj_round'] = pd.to_numeric(draft_picks_df['adj_round'].fillna(draft_picks_df['round']), downcast='integer')
draft_picks_df['years_kept'] = np.where(draft_picks_df['round'] == draft_picks_df['adj_round'], 0, 1)
draft_picks_df['base_salary'] = (75 - (5 * (draft_picks_df['adj_round'] - 1)))

In [193]:
draft_picks_df

,player_id,round,manager_id,is_keeper,adj_round,years_kept,base_salary
0,9509,1,992614081347579904,None,1,0,75
1,1466,1,870168774161178624,None,1,0,75
2,3198,1,990773448974618624,None,1,0,75
3,4988,1,988913995279036416,None,1,0,75
4,7525,1,988914274368020480,None,1,0,75
...,...,...,...,...,...,...,...
145,7547,15,988915873790459904,True,5,1,55
146,4881,15,988915756895113216,True,5,1,55
147,4663,15,985202843810332672,True,1,1,75
148,4039,15,855933294594879488,True,1,1,75


In [198]:
rosterd_players_df = sqldf("""
SELECT
    players_df.player_id
    ,players_df.first_name
    ,players_df.last_name
    ,players_df.position
    ,Coalesce(draft_picks_df.base_salary, 5) AS base_salary
    ,Coalesce(draft_picks_df.years_kept,0) AS years_kept
    ,managers_df.user_id
    ,managers_df.display_name
    FROM players_df
    LEFT JOIN curr_rosters_df
        ON curr_rosters_df.players = players_df.player_id
    LEFT JOIN managers_df
        ON managers_df.user_id = curr_rosters_df.manager_id
    LEFT JOIN draft_picks_df
        ON draft_picks_df.player_id = players_df.player_id
    WHERE players_df.player_id IN (SELECT players FROM curr_rosters_df)
""")

In [199]:
rosterd_players_df

,player_id,first_name,last_name,position,base_salary,years_kept,user_id,display_name
0,10222,Jayden,Reed,WR,5,0,990773448974618624,RyJohn
1,10229,Rashee,Rice,WR,5,0,855933294594879488,rmasons
2,10236,Dalton,Kincaid,TE,45,0,988913995279036416,hatersandlosers
3,10859,Sam,LaPorta,TE,25,0,865123387251519488,aspear3
4,10937,Jake,Moody,K,5,0,855933294594879488,rmasons
...,...,...,...,...,...,...,...,...
150,LAC,Los Angeles,Chargers,DEF,5,0,990773448974618624,RyJohn
151,NYJ,New York,Jets,DEF,30,0,988915873790459904,GriffDaGreat
152,PHI,Philadelphia,Eagles,DEF,35,0,870168774161178624,kerrveball
153,PIT,Pittsburgh,Steelers,DEF,15,0,865123387251519488,aspear3


In [183]:
rosterd_players_df.to_excel('tds_2023.xlsx', index=False)

In [94]:

current_week, current_season, nfl_state = getNflState()
transactions = {}
player_list = []
matchup_list = []
for wk in range(1, current_week):
    transactions[str(wk)] = getTransactions(league_id, str(wk))
    player_week_list, matchup_week_list = getMatchups(league_id, str(wk))
    player_list.extend(player_week_list)
    matchup_list.extend(matchup_week_list)


In [264]:
league_info = getLeagueInfo(league_id=league_id)

In [265]:
league_info

{'984533715101495296': {'total_teams': 10,
  'roster_poositions': ['QB',
   'RB',
   'RB',
   'WR',
   'WR',
   'TE',
   'FLEX',
   'FLEX',
   'K',
   'DEF',
   'BN',
   'BN',
   'BN',
   'BN',
   'BN'],
  'draft_id': '995877095513481216',
  'season': '2023',
  'league_name': "Show Us Your TD's",
  'playoff_start_wk': 15,
  'num_playoff_teams': 6,
  'faab_budget': 200}}

In [254]:
rosters, rosters_df = getCurrentRosters(league_id)

In [270]:
rosters_df['longest_win_streak'] = rosters_df['record'].apply(longest_win_streak)
rosters_df['longest_loss_streak'] = rosters_df['record'].apply(longest_loss_streak)

In [176]:
player_info_df = getPlayers()

In [175]:
player_df = pd.DataFrame(player_list, columns=['player_id', 'roster_id', 'week', 'is_starter', 'points'])
matchup_df = pd.DataFrame(matchup_list, columns=['matchup_id', 'week', 'roster_id'])

In [177]:
player_df = ps.sqldf("""
SELECT
    pdf.*
    ,pi.first_name
    ,pi.last_name
    ,pi.is_rookie
    ,pi.position
    FROM player_df pdf
    LEFT JOIN player_info_df pi
        ON pi.player_id = pdf.player_id          
""")

In [179]:
weekly_pos_rnks = ps.sqldf("""
SELECT
    roster_id
    ,week
    ,player_id
    ,position
    ,points
    ,Rank() OVER (PARTITION BY roster_id, week, position ORDER BY points DESC, player_id) AS pos_rnk
    FROM player_df
    WHERE 1=1
""")

In [180]:
top_weekly_players = ps.sqldf("""
SELECT
    roster_id
    ,week
    ,player_id
    FROM
        (SELECT
            roster_id
            ,week
            ,player_id
            ,position
            ,points
            ,Rank() OVER (PARTITION BY roster_id, week ORDER BY points DESC, player_id) AS flex_rnk
            FROM weekly_pos_rnks
            WHERE 1=1
                AND ((position IN ('WR', 'RB') AND pos_rnk > 2) OR (position = 'TE' AND pos_rnk > 1))   
        )
    WHERE flex_rnk < 3
UNION ALL
SELECT
    roster_id
    ,week
    ,player_id
    FROM weekly_pos_rnks
    WHERE 1=1
        AND ((position NOT IN ('WR', 'RB') AND pos_rnk = 1) OR (position IN ('WR', 'RB') AND pos_rnk < 3))
""")

In [181]:
player_df = ps.sqldf("""
SELECT
    fp.*
    ,CASE
        WHEN tp.player_id IS NOT NULL
            THEN True
        ELSE False
    END AS top_player
    FROM player_df fp
    LEFT JOIN top_weekly_players tp
        ON tp.player_id = fp.player_id
        AND tp.week = fp.week
        AND tp.roster_id = fp.roster_id
""")

In [354]:
starters_by_score = """
SELECT
    player_id
    ,first_name
    ,last_name
    ,Sum(points) AS ttl_strtd_points
    FROM player_df
    WHERE 1=1
        AND is_starter = 1
    GROUP BY 1,2,3
    ORDER BY 4 DESC
"""

In [359]:
ps.sqldf(starters_by_score).head(25)

,player_id,first_name,last_name,ttl_strtd_points
0,4984,Josh,Allen,432.58
1,4034,Christian,McCaffrey,402.30
2,6786,CeeDee,Lamb,381.70
3,3321,Tyreek,Hill,379.20
4,6904,Jalen,Hurts,355.72
5,7547,Amon-Ra,St. Brown,311.50
6,6768,Tua,Tagovailoa,307.86
7,5859,A.J.,Brown,299.70
8,4881,Lamar,Jackson,295.64
9,1479,Keenan,Allen,288.86


In [353]:
player_df

,player_id,roster_id,week,is_starter,points,first_name,last_name,is_rookie,position,top_player
0,WAS,1,1,1,16.80,Washington,Commanders,0,DEF,1
1,9997,1,1,0,17.70,Zay,Flowers,1,WR,1
2,9753,1,1,1,1.10,Zach,Charbonnet,1,RB,0
3,96,1,1,1,0.00,Aaron,Rodgers,0,QB,1
4,9221,1,1,1,8.00,Jahmyr,Gibbs,1,RB,1
...,...,...,...,...,...,...,...,...,...,...
2631,4984,10,17,1,21.16,Josh,Allen,0,QB,1
2632,4234,10,17,0,1.80,Noah,Brown,0,WR,0
2633,4199,10,17,1,15.00,Aaron,Jones,0,RB,1
2634,4037,10,17,1,18.10,Chris,Godwin,0,WR,1


In [258]:
rookies_by_manager_sql = """
SELECT
    rm.owner_name
    ,count(distinct player_id) AS rookies_started
    FROM player_df pdf
    LEFT JOIN roster_manager_df rm
        ON rm.roster_id = pdf.roster_id
    WHERE 1=1
        AND is_rookie = 1
        AND is_starter = 1
    GROUP BY 1
    ORDER BY rookies_started DESC
"""

In [259]:
ps.sqldf(rookies_by_manager_sql)

,owner_name,rookies_started
0,Mason,9
1,Uriah,7
2,Jared,4
3,Spencer,2
4,Ryan,2
5,Conner,2
6,Catherine,1
7,Andrew,1


In [191]:
manager_yearly_points = ps.sqldf("""
SELECT
    roster_id
    ,Sum(CASE
        WHEN is_Starter = 1
            THEN points
        ELSE 0
    END) AS starter_points
    ,Sum(CASE
        WHEN top_player = 1
            THEN points
        ELSE 0
    END) AS top_points
    FROM player_df
    GROUP BY 1
""")

In [260]:
mngr_effcncy_sql = """
SELECT
    rm.owner_name
    ,(starter_points/top_points) * 100 AS effcncy
    FROM manager_yearly_points myp
    LEFT JOIN roster_manager_df rm
        ON rm.roster_id = myp.roster_id
    ORDER BY effcncy DESC
"""

In [261]:
ps.sqldf(mngr_effcncy_sql)

,owner_name,effcncy
0,Madelyn,91.935108
1,Andrew,90.066564
2,Conner,89.641795
3,Spencer,89.524343
4,Catherine,89.172353
5,Mason,89.077596
6,Uriah,88.228681
7,Jared,87.918856
8,Ryan,86.812853
9,Christian,86.128488


In [278]:
reg_season_avg_scores_sql = """
SELECT
    rm.owner_name
    ,Sum(pdf.points) as ttl_points
    FROM player_df pdf
    LEFT JOIN roster_manager_df rm
        ON rm.roster_id = pdf.roster_id
    WHERE 1=1
        AND pdf.is_starter = 1
        AND pdf.week < 15
    GROUP BY 1
    ORDER BY 2 DESC
"""

In [279]:
ps.sqldf(reg_season_avg_scores_sql)

,owner_name,ttl_points
0,Jared,2054.94
1,Madelyn,2046.08
2,Andrew,2031.32
3,Conner,1941.80
4,Mason,1934.56
5,Christian,1912.94
6,Ryan,1874.62
7,Catherine,1871.04
8,Uriah,1819.64
9,Spencer,1755.84


In [321]:
mngr_wkly_pnts = """
SELECT
    roster_id
    ,week
    ,sum(points) AS ttl_points
    FROM player_df
    WHERE 1=1
        AND is_starter =1 
    GROUP BY 1,2
"""

In [324]:
mngr_wkly = ps.sqldf(mngr_wkly_pnts)

In [339]:
mtchps_sql = """
SELECT
    a.week
    ,CASE
        WHEN a.roster_id < b.roster_id
            THEN a.roster_id
        ELSE b.roster_id
    END AS mngr_a
    ,CASE
        WHEN a.roster_id > b.roster_id
            THEN a.roster_id
        ELSE b.roster_id
    END AS mngr_b
    FROM matchup_df a
    LEFT JOIN matchup_df b
        ON b.matchup_id = a.matchup_id
        AND b.roster_id != a.roster_id
        AND b.week = a.week
    WHERE 1=1
        AND b.roster_id IS NOT NULL
    GROUP BY 1,2,3
"""

In [340]:
mtchups_df = ps.sqldf(mtchps_sql)

In [378]:
mtchp_pnts_sql = """
SELECT
    m.week
    ,rma.owner_name AS owner_a
    ,a.ttl_points AS mngr_a_points
    ,rmb.owner_name AS owner_b
    ,b.ttl_points AS mngr_b_points
    ,CASE
        WHEN a.ttl_points > b.ttl_points
            THEN rma.owner_name
        ELSE rmb.owner_name
    END AS winner
    ,CASE
        WHEN a.ttl_points > b.ttl_points
            THEN a.ttl_points - b.ttl_points
        ELSE b.ttl_points - a.ttl_points
    END AS pnt_diff
    FROM mtchups_df m
    LEFT JOIN mngr_wkly a
        ON a.roster_id = m.mngr_a
        AND a.week = m.week
    LEFT JOIN mngr_wkly b
        ON b.roster_id = m.mngr_b
        AND b.week = m.week
    LEFT JOIN roster_manager_df rma
        ON rma.roster_id = m.mngr_a
    LEFT JOIN roster_manager_df rmb
        ON rmb.roster_id = m.mngr_b
    ORDER BY 7 DESC
"""

In [379]:
matchup_ttls_df = ps.sqldf(mtchp_pnts_sql)

In [389]:
ps.sqldf('SELECT * FROM matchup_ttls_df WHERE mngr_b_points < 111.62 OR mngr_a_points < 111.62')

,week,owner_a,mngr_a_points,owner_b,mngr_b_points,winner,pnt_diff
0,8,Jared,213.82,Ryan,102.20,Jared,111.62
1,4,Conner,194.88,Catherine,103.48,Conner,91.40
2,5,Spencer,102.62,Jared,179.32,Jared,76.70
3,13,Conner,77.88,Catherine,153.72,Catherine,75.84
4,16,Conner,165.52,Ryan,92.88,Conner,72.64
5,4,Ryan,106.30,Uriah,178.20,Uriah,71.90
6,9,Mason,68.88,Andrew,130.94,Andrew,62.06
7,9,Catherine,106.78,Jared,164.92,Jared,58.14
8,11,Catherine,168.28,Madelyn,111.26,Catherine,57.02
9,3,Jared,159.96,Uriah,103.52,Jared,56.44


In [380]:
matchup_ttls_df

,week,owner_a,mngr_a_points,owner_b,mngr_b_points,winner,pnt_diff
0,8,Jared,213.82,Ryan,102.20,Jared,111.62
1,4,Conner,194.88,Catherine,103.48,Conner,91.40
2,5,Madelyn,124.84,Andrew,214.66,Andrew,89.82
3,5,Spencer,102.62,Jared,179.32,Jared,76.70
4,13,Ryan,193.10,Uriah,116.96,Ryan,76.14
...,...,...,...,...,...,...,...
76,10,Catherine,93.50,Ryan,97.74,Ryan,4.24
77,15,Conner,159.14,Spencer,155.12,Conner,4.02
78,6,Andrew,106.84,Ryan,110.34,Ryan,3.50
79,1,Mason,96.40,Madelyn,93.26,Mason,3.14


In [384]:
lrgst_mrgn_vctry = """
SELECT
    winner
    ,sum(pnt_diff) / count(week) AS avg_vctry
    FROM matchup_ttls_df
    GROUP BY 1
"""

In [385]:
ps.sqldf(lrgst_mrgn_vctry)

,winner,avg_vctry
0,Andrew,47.068889
1,Catherine,31.672500
2,Christian,32.080000
3,Conner,34.360000
4,Jared,46.006000
5,Madelyn,36.394000
6,Mason,35.011111
7,Ryan,26.033333
8,Spencer,26.713333
9,Uriah,38.476667


In [283]:
ps.sqldf(mngr_wkly_pnts)

,roster_id,week,ttl_points
0,1,1,96.40
1,1,2,157.44
2,1,3,125.14
3,1,4,118.90
4,1,5,123.36
...,...,...,...
165,10,13,116.96
166,10,14,96.82
167,10,15,122.86
168,10,16,117.78


In [281]:
matchup_df.head()

,matchup_id,week,roster_id
0,2.0,1,1
1,3.0,1,2
2,1.0,1,3
3,4.0,1,4
4,2.0,1,5
